# Data Cleanup and Analysis Code

In [1]:
import pandas as pd

# Transform

### Table 1 : 'US_States'

In [2]:
states_df =  pd.read_csv('Resources/Source_Data/US_States.csv')
states_df

#Trim leading and trailing spaces for string type data
sdf_obj = states_df.select_dtypes(['object'])
states_df[sdf_obj.columns] = sdf_obj.apply(lambda x: x.str.strip()) 


# LOAD US_States.csv

In [3]:
# Check for duplicates 
# checking total States_Fips vs. total unique States_Fips. 
# If they are equal, then there are no duplicates
States_Fips_List = states_df['State_Fips']
States_Table_Count = States_Fips_List.count()
Unique_States_Count = States_Fips_List.nunique()
print (States_Table_Count, Unique_States_Count)

# State Table is clean. No duplicate FIPS exist. 
# This table is ready for PostgreSQL Table


60 60


In [4]:

# Write to US_States.csv that can be imported in PostgreSQL
states_df.to_csv('Resources/Transformed_Data/Us_States.csv', index=False)

# Transform

### Table 2 : 'US_Counties'

In [5]:
counties_df = pd.read_csv('Resources/Source_Data/US_Counties.csv', encoding='latin-1')

# counties are uniquly identified by County_FIPS which is a unique ID 
# called Federal Information Processing Standards
# Check for duplicates 
# I am checking total County_Fips vs. total unique County_Fips. 
# If they are equal, then there are no duplicates
County_Fips_List = counties_df['County_Fips']
County_Table_Count = County_Fips_List.count()
Unique_Counties_Count = County_Fips_List.nunique()
print (County_Table_Count, Unique_Counties_Count)


# County Table is Clean. No Nulls, no duplicate FIPS exist
# Trim leading and trailing spaces for string type data
df_obj = counties_df.select_dtypes(['object'])
counties_df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()) 

# Merge County table with state 
county_table = counties_df.merge(states_df, how='left', left_on='State', right_on='Sabbr')
county_table = county_table[['County_Fips', 'County', 'State_Fips']]
county_table['County'] = county_table['County'].str.title()


df_obj = county_table.select_dtypes(['object'])
county_table[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()) 
county_table_strip = county_table
county_table_strip

# This table is ready for PostGressql
county_table_strip
# LOAD Us_Counties.csv

# Write to Us_Counties.csv that can be imported in PostgreSQL
county_table_strip.to_csv('Resources/Transformed_Data/Us_Counties.csv', index=False)

# TRANSFORM

### Table 3 : 'US_Covid_Data'

In [11]:
# covid_df = pd.read_csv('Resources/Source_Data/US_Covid_Data.csv', parse_dates=['date'])
# covid_df

# This will connect to the raw file on their website to get yo to date data
covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv', parse_dates=['date'])
#if we are reading it form the raw file from the source, we can write it to csv if we choose to save it locally
#covid_df.to_csv('Resources/Source_Data/US_Covid_Data.csv', index=False)
covid_df
# Chack for Nulls in any colums
covid_df.isna().any()
# column 'deaths' can contain Nulls but not Fips 
# Identify these records for cleaning

# if (covid_df['fips'].isnull().values.any()):
#     print(covid_df[covid_df['fips'].isna()])

covid_df['fips'].isnull().values.any()
# If covid table has records that have state but an unknown county, we keep the data
na_covid_df = covid_df[covid_df['fips'].isna()]
#na_covid_df
# stripping 'state' column of any leading and trailing spaces

#na_covid_df['state'] = na_covid_df['state'].str.strip()
na_covid_df['state'].apply(lambda x: x.strip()) 
#na_covid_df

na_covid_df = na_covid_df.merge(county_table_strip, how="left", left_on = "state", right_on="County")
#na_covid_df

na_covid_df_final = na_covid_df[['date', 'county', 'state', 'County_Fips', 'cases', 'deaths']]
na_covid_df_final

# Records with Null Fips have been cleaned and fips from cencus table for --
# -- state_unknown counties have been used
# Renaming the column to match Us_Covid_Table

na_covid_df_final = na_covid_df_final.rename(columns = {'County_Fips':'county_fips'})
na_covid_df_final

na_covid_df_final['county_fips'].isnull().values.any()

# Non Null Covid Data
non_na_covid_df = covid_df[covid_df['fips'].notna()]  
non_na_covid_df

non_na_covid_df = non_na_covid_df.rename(columns = {'fips':'county_fips'})
non_na_covid_df
# Concatenate the clean dataframes
vertical_stack = pd. concat([na_covid_df_final, non_na_covid_df], axis=0) 
vertical_stack.isnull().any()
vertical_stack['deaths'] = vertical_stack['deaths'].fillna(0)
# Converting fips columns to Int
vertical_stack.county_fips = vertical_stack.county_fips.astype(int)
vertical_stack.deaths = vertical_stack.deaths.astype(int)


us_covid_table = vertical_stack[['date', 'county_fips', 'cases', 'deaths']]
#us_covid_table = us_covid_table.sort_values(by='date', ascending=False)
us_covid_table

# This table is ready for Loading
# LOAD US_Covid_Data.csv

# Write to US_Covid_Data.csv that can be imported in PostgreSQL
us_covid_table.to_csv('Resources/Transformed_Data/US_Covid_Data.csv', index=False)
# Transform

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1598913,2021-08-08,Sweetwater,Wyoming,56037.0,5056,45.0
1598914,2021-08-08,Teton,Wyoming,56039.0,3960,11.0
1598915,2021-08-08,Uinta,Wyoming,56041.0,2498,14.0
1598916,2021-08-08,Washakie,Wyoming,56043.0,949,26.0


### Table 4 : 'US_Census_Data_2020'

In [22]:
population_data = pd.read_csv('Resources/Source_Data/US_Census_Data_2020.csv', encoding='latin-1')
# population_data.loc[population_data['STNAME'] == population_data['CTYNAME']]
# County = 0 are records for State Totals
# removed these because state totals can be calculated from county information
population_data = population_data.loc[population_data['COUNTY'] > 0]

# converted state and county fields from int to str
population_data['STATE'] = population_data['STATE'].astype('str')
population_data['COUNTY'] = population_data['COUNTY'].astype('str')

# filled with leading zeros to get the format required for fips_county
population_data['STATE']=population_data['STATE'].apply(lambda x: x.zfill(2))
population_data['COUNTY']=population_data['COUNTY'].apply(lambda x: x.zfill(3))

# concatenated state and county fips to make a County_Fips Code that follows the Fips_County convention
population_data['fips'] = population_data['STATE'] + population_data['COUNTY']
# population_data['fips'] 
population_data


population_data = population_data[['fips','POPESTIMATE2016','POPESTIMATE2017','POPESTIMATE2018','POPESTIMATE2019',
'POPESTIMATE2020', 'BIRTHS2016','BIRTHS2017','BIRTHS2018','BIRTHS2019','BIRTHS2020',
'DEATHS2016','DEATHS2017','DEATHS2018','DEATHS2019','DEATHS2020']]
population_data
# Table is ready to load

# LOAD US_Census_Data.csv

# Write to US_Census_Data.csv that can be imported in PostgreSQL
population_data.to_csv('Resources/Transformed_Data/US_Census_Data.csv', index=False)

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019,RNETMIG2020
1,50,3,6,1,1,Alabama,Autauga County,54571,54582,54761,...,6.236931,-5.971016,-3.773344,2.206640,-1.529706,4.954403,0.993228,-0.018021,3.486011,6.290545
2,50,3,6,1,3,Alabama,Baldwin County,182265,182263,183121,...,16.705437,17.670696,22.924288,20.300088,17.902273,21.436499,22.476720,24.846335,25.242507,26.401562
3,50,3,6,1,5,Alabama,Barbour County,27457,27454,27325,...,0.329254,-6.860371,-8.093425,-5.063857,-15.677998,-18.377839,-25.138734,-8.790155,-6.257064,0.649799
4,50,3,6,1,7,Alabama,Bibb County,22915,22904,22858,...,-4.912927,-3.789130,-5.800695,1.420612,1.286202,-0.841769,-3.235672,-7.271592,0.268980,-7.199262
5,50,3,6,1,9,Alabama,Blount County,57322,57322,57372,...,0.348029,-1.597971,-0.277742,-1.997117,-1.303543,-1.217158,6.193186,0.242275,0.934175,1.192544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3189,50,4,8,56,37,Wyoming,Sweetwater County,43806,43806,43580,...,0.776433,15.410190,-4.433558,-12.751566,-13.455712,-17.688190,-20.936101,-15.589918,-10.452355,-9.510457
3190,50,4,8,56,39,Wyoming,Teton County,21294,21298,21298,...,-2.340824,2.322071,23.284369,12.672811,4.881876,1.035867,-1.543805,-13.120659,0.171505,0.383943
3191,50,4,8,56,41,Wyoming,Uinta County,21118,21121,21090,...,-17.908599,-4.151853,-10.624866,-15.022486,-10.381621,-11.424990,-18.658892,-14.135663,-8.840598,-2.177625
3192,50,4,8,56,43,Wyoming,Washakie County,8533,8528,8531,...,-12.837122,-3.084040,-1.307423,-19.048760,0.000000,-15.064998,-16.056321,-16.101642,-7.638447,-6.801848


# Transform

### Table 5 : 'WHO_Flu_data'

In [26]:
who_flu_df = pd.read_csv('Resources/Source_Data/WHO_NREVSS_Clinical_Labs.csv')
#who_flu_df

# Drop Region Type
who_flu_df.drop(columns=['REGION TYPE'], inplace=True) 

# Replace 'X' with 0
who_flu_df.loc[who_flu_df['REGION'] == 'New York City', 'REGION'] = 'New York'
who_flu_df.loc[who_flu_df['TOTAL SPECIMENS'] == 'X', 'TOTAL SPECIMENS'] = 0
who_flu_df.loc[who_flu_df['TOTAL A'] == 'X', 'TOTAL A'] = 0
who_flu_df.loc[who_flu_df['TOTAL B'] == 'X', 'TOTAL B'] = 0
who_flu_df.loc[who_flu_df['PERCENT POSITIVE'] == 'X', 'PERCENT POSITIVE'] = 0
who_flu_df.loc[who_flu_df['PERCENT A'] == 'X', 'PERCENT A'] = 0
who_flu_df.loc[who_flu_df['PERCENT B'] == 'X', 'PERCENT B'] = 0
who_flu_df

,REGION,YEAR,WEEK,TOTAL SPECIMENS,TOTAL A,TOTAL B,PERCENT POSITIVE,PERCENT A,PERCENT B
0,Alabama,2019,40,512,2,13,2.93,0.39,2.54
1,Alaska,2019,40,0,0,0,0,0,0
2,Arizona,2019,40,278,0,2,0.72,0,0.72
3,Arkansas,2019,40,89,0,0,0,0,0
4,California,2019,40,1776,18,10,1.58,1.01,0.56
...,...,...,...,...,...,...,...,...,...
5125,Wisconsin,2021,29,0,0,0,0,0,0
5126,Wyoming,2021,29,0,0,0,0,0,0
5127,Puerto Rico,2021,29,0,0,0,0,0,0
5128,Virgin Islands,2021,29,0,0,0,0,0,0


In [27]:
# Add State_Fips and Remove Region
# Strip spaced
who_flu_df['REGION'].apply(lambda x: x.strip()) 

# Merge with US_States
who_flu_table = who_flu_df.merge(states_df, how='left', left_on='REGION', right_on='Sname')
who_flu_table = who_flu_table[['State_Fips','YEAR','WEEK','TOTAL SPECIMENS','TOTAL A','TOTAL B','PERCENT POSITIVE',
'PERCENT A','PERCENT B']]
who_flu_table
# Table is ready to load

,State_Fips,YEAR,WEEK,TOTAL SPECIMENS,TOTAL A,TOTAL B,PERCENT POSITIVE,PERCENT A,PERCENT B
0,1,2019,40,512,2,13,2.93,0.39,2.54
1,2,2019,40,0,0,0,0,0,0
2,4,2019,40,278,0,2,0.72,0,0.72
3,5,2019,40,89,0,0,0,0,0
4,6,2019,40,1776,18,10,1.58,1.01,0.56
...,...,...,...,...,...,...,...,...,...
5125,55,2021,29,0,0,0,0,0,0
5126,56,2021,29,0,0,0,0,0,0
5127,72,2021,29,0,0,0,0,0,0
5128,78,2021,29,0,0,0,0,0,0


# LOAD WHO_Flu_Data.csv

In [28]:
# Write to WHO_Flu_Data.csv that can be imported in PostgreSQL
who_flu_table.to_csv('Resources/Transformed_Data/WHO_Flu_Data.csv', index=False)